<a href="https://colab.research.google.com/github/UW-CTRL/AA548-spr2024/blob/main/demo/trajopt_shooting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp


In [ ]:
dt = 0.1
# state: [x, y, vx, vy]
# control: [ax, ay]
A_dynamics = np.array([[1., 0., dt, 0.],
                        [0., 1., 0., dt],
                        [0., 0., 1., 0.],
                        [0., 0., 0., 1.]])

B_dynamics = np.array([[0.5 * dt**2, 0.],
                        [0., 0.5 * dt**2],
                        [dt, 0.],
                        [0., dt]])

## Shooting method

In [ ]:
T = 20  # time steps
n = 4   # state dimension
m = 2   # control dimension

us = cp.Variable([T,m])   # optimization variable.

# quadratic cost matrices (play around with these numbers)
Q = np.diag([1., 2., 0., 2.])
Qt = np.eye(n) * 5.0
R = np.diag([0., 2.])
discount = 1.0   # you can add a discount or markup to make running cost time dependent

goal_state = np.zeros(n)
initial_state = np.array([-4, 0., 0., 2.])
state = initial_state
u_max = 2.


objective = 0
constraints = []

for t in range(T):
    objective += discount**t * (cp.quad_form(state, Q) + cp.quad_form(us[t], R))
    state = A_dynamics @ state + B_dynamics @ us[t]
    constraints += [cp.norm(us[t], 2) <= u_max]

# add or remove goal_state cost or goal_state constraint
objective += cp.quad_form(state - goal_state, Qt)
# constraints += [state == goal_state]

problem = cp.Problem(cp.Minimize(objective), constraints)
problem.solve(verbose=True)

In [ ]:
states = [initial_state]
for t in range(T):
    states.append(A_dynamics @ states[t] + B_dynamics @ us.value[t])
states = np.stack(states)

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(states[:,0], states[:,1])
plt.quiver(states[:,0], states[:,1], states[:,2], states[:,3], scale=30, width=.003)
plt.quiver(states[:-1,0], states[:-1,1], us.value[:,0], us.value[:,1], scale=30, width=.003, color='red')
plt.title("Optimal Trajectory with velocity and acceleration vector")
plt.xlabel("$x$ position")
plt.ylabel("$y$ position")
plt.grid()

plt.subplot(1,2,2)
plt.plot(us.value)
plt.plot(np.linalg.norm(us.value, 2, axis=1), '--')
plt.title("Optimal Control Sequence")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration")
plt.legend(["$a_x$", "$a_y$", "$\|a\|_2$"])
plt.grid()